In [3]:
import json
import pprint
from bson.objectid import ObjectId

from db_connection import get_client

In [37]:
client = get_client()
printer = pprint.PrettyPrinter()
person_collection = client.production.person_collection

Connection to MongoDB is succesful!


In [44]:
def insert_json_data(path, collection):
  with open(path, 'r') as file:
    data = json.load(file)
    result = collection.insert_many(data)
  print('Inserted document amount: ', len(result.inserted_ids))
  print('Inserted document IDs:', result.inserted_ids)

insert_json_data('data/person_data.json', person_collection)

Inserted document amount:  10
Inserted document IDs: [ObjectId('67845b56e5cb4a4371b64352'), ObjectId('67845b56e5cb4a4371b64353'), ObjectId('67845b56e5cb4a4371b64354'), ObjectId('67845b56e5cb4a4371b64355'), ObjectId('67845b56e5cb4a4371b64356'), ObjectId('67845b56e5cb4a4371b64357'), ObjectId('67845b56e5cb4a4371b64358'), ObjectId('67845b56e5cb4a4371b64359'), ObjectId('67845b56e5cb4a4371b6435a'), ObjectId('67845b56e5cb4a4371b6435b')]


In [74]:
def insert_test_doc(client):
  collection = client.test_db.test_col
  test_document = {
    "name": "some_doc",
    "type": "test"
  }
  inserted_doc = collection.insert_one(test_document)
  result = collection.find_one({"_id": inserted_doc.inserted_id})
  print(f"Inserted document: {result}")

insert_test_doc(client)

Inserted document: {'_id': ObjectId('67844454c074166f01b78686'), 'name': 'some_doc', 'type': 'test'}


In [93]:
def create_documents(collection):
  first_names = ["Jan", "Jennifer", "Alice", "Bob", "Catherine", "David", "Emma", "Frank", "Grace"]
  last_names = ["Bart", "Pit", "Smith", "Johnson", "Williams", "Brown", "Jones", "Garcia", "Martinez"]
  ages = [30, 10, 40, 22, 35, 28, 18, 50, 45]

  docs = []
  for first_name, last_name, age in zip(first_names, last_names, ages):
    doc = {"first_name": first_name, "last_name": last_name, "age": age}
    docs.append(doc)
  collection.insert_many(docs)

  for doc in person_collection.find():
    print(doc)

create_documents(person_collection)

{'_id': ObjectId('6784486cc074166f01b7869f'), 'first_name': 'Jan', 'last_name': 'Bart', 'age': 30}
{'_id': ObjectId('6784486cc074166f01b786a0'), 'first_name': 'Jennifer', 'last_name': 'Pit', 'age': 10}
{'_id': ObjectId('6784486cc074166f01b786a1'), 'first_name': 'Alice', 'last_name': 'Smith', 'age': 40}
{'_id': ObjectId('6784486cc074166f01b786a2'), 'first_name': 'Bob', 'last_name': 'Johnson', 'age': 22}
{'_id': ObjectId('6784486cc074166f01b786a3'), 'first_name': 'Catherine', 'last_name': 'Williams', 'age': 35}
{'_id': ObjectId('6784486cc074166f01b786a4'), 'first_name': 'David', 'last_name': 'Brown', 'age': 28}
{'_id': ObjectId('6784486cc074166f01b786a5'), 'first_name': 'Emma', 'last_name': 'Jones', 'age': 18}
{'_id': ObjectId('6784486cc074166f01b786a6'), 'first_name': 'Frank', 'last_name': 'Garcia', 'age': 50}
{'_id': ObjectId('6784486cc074166f01b786a7'), 'first_name': 'Grace', 'last_name': 'Martinez', 'age': 45}
{'_id': ObjectId('6784495bc074166f01b786a8'), 'first_name': 'Jan', 'last_n

In [33]:
def find_all_ents(collection):
  people = collection.find().limit(3)

  for person in people:
    printer.pprint(person)
find_all_ents(person_collection)

{'_id': ObjectId('6784486cc074166f01b7869f'),
 'age': 30,
 'first_name': 'Jan',
 'last_name': 'Bart'}
{'_id': ObjectId('6784486cc074166f01b786a0'),
 'age': 10,
 'first_name': 'Jennifer',
 'last_name': 'Pit'}
{'_id': ObjectId('6784486cc074166f01b786a1'),
 'age': 40,
 'first_name': 'Alice',
 'last_name': 'Smith'}


In [ ]:
martinez = {"last_name": "Martinez"}

def find_entity(collection, query):
  entity = collection.find_one(query)
  printer.pprint(entity)

find_entity(person_collection, martinez)

{'_id': ObjectId('6784486cc074166f01b786a7'),
 'age': 45,
 'first_name': 'Grace',
 'last_name': 'Martinez'}


In [103]:
def count_entities(collection):
  count = collection.count_documents(filter={})
  print(f"Number of People: {count}")
count_entities(person_collection)

Number of People: 18


In [31]:
bob_id = '6784486cc074166f01b786a2'

def find_one(id):
  _id = ObjectId(id)
  person= person_collection.find_one({"_id": _id})

  if person:
    printer.pprint(person)
  else:
    print("No person found with a given ID.")

find_one(bob_id)

{'_id': ObjectId('6784486cc074166f01b786a2'),
 'age': 22,
 'first_name': 'Bob',
 'last_name': 'Johnson'}


In [11]:
def get_age_range(collection, min_age, max_age):
  query = {"$and": [
      {"age": {"$gte": min_age}},
      {"age": {"$lte": max_age}}
    ]}

  filtered_people = collection.find(query).sort("age")
  print(f"People between {min_age}-{max_age} y.o:\n")
  for person in filtered_people:
    printer.pprint(person)

get_age_range(person_collection, min_age=30, max_age=40)

People between 30-40 y.o:

{'_id': ObjectId('6784486cc074166f01b7869f'),
 'age': 30,
 'first_name': 'Jan',
 'last_name': 'Bart'}
{'_id': ObjectId('6784495bc074166f01b786a8'),
 'age': 30,
 'first_name': 'Jan',
 'last_name': 'Bart'}
{'_id': ObjectId('6784486cc074166f01b786a3'),
 'age': 35,
 'first_name': 'Catherine',
 'last_name': 'Williams'}
{'_id': ObjectId('6784495bc074166f01b786ac'),
 'age': 35,
 'first_name': 'Catherine',
 'last_name': 'Williams'}
{'_id': ObjectId('6784486cc074166f01b786a1'),
 'age': 40,
 'first_name': 'Alice',
 'last_name': 'Smith'}
{'_id': ObjectId('6784495bc074166f01b786aa'),
 'age': 40,
 'first_name': 'Alice',
 'last_name': 'Smith'}


In [ ]:
person_projection = {"_id":0, "first_name": 1, "last_name": 1}

def project_columns(query, collection):
  filtered_people = collection.find({}, query)

  for person in filtered_people:
    printer.pprint(person)

print("Person projection without ID:\n")
project_columns(person_projection, person_collection)

Person projection without ID:

{'first_name': 'Jan', 'last_name': 'Bart'}
{'first_name': 'Jennifer', 'last_name': 'Pit'}
{'first_name': 'Alice', 'last_name': 'Smith'}
{'first_name': 'Bob', 'last_name': 'Johnson'}
{'first_name': 'Catherine', 'last_name': 'Williams'}
{'first_name': 'David', 'last_name': 'Brown'}
{'first_name': 'Emma', 'last_name': 'Jones'}
{'first_name': 'Frank', 'last_name': 'Garcia'}
{'first_name': 'Grace', 'last_name': 'Martinez'}
{'first_name': 'Jan', 'last_name': 'Bart'}
{'first_name': 'Jennifer', 'last_name': 'Pit'}
{'first_name': 'Alice', 'last_name': 'Smith'}
{'first_name': 'Bob', 'last_name': 'Johnson'}
{'first_name': 'Catherine', 'last_name': 'Williams'}
{'first_name': 'David', 'last_name': 'Brown'}
{'first_name': 'Emma', 'last_name': 'Jones'}
{'first_name': 'Frank', 'last_name': 'Garcia'}
{'first_name': 'Grace', 'last_name': 'Martinez'}


In [30]:
update_query = {
    "$set": {"new_field": True},
    "$inc": {"age": 1},
    "$rename": {"first_name": "firstName", "last_name": "lastName"}
  }

def update_entity_by_id(id, collection, query):
  _id = ObjectId(id)

  collection.update_one({"_id": _id}, query)
  updated_ent = collection.find_one({"_id": _id})
  print(f"Updated entitiy:\n{updated_ent}")

update_entity_by_id('6784495bc074166f01b786a9', person_collection, update_query)

Updated entitiy:
{'_id': ObjectId('6784495bc074166f01b786a9'), 'age': 13, 'firstName': 'Jennifer', 'lastName': 'Pit', 'new_field': True}


In [36]:
doc_gandalf = {
    "first_name": "Gandalf",
    "last_name": "the Grey",
    "age": 24000
  }

def replace_one(id, collection, new_doc):
  _id = ObjectId(id)
  collection.replace_one({"_id": _id}, new_doc)
  updated_doc = collection.find_one({"_id": _id})
  printer.pprint(updated_doc)

replace_one('6784495bc074166f01b786b0', person_collection, doc_gandalf)


{'_id': ObjectId('6784495bc074166f01b786b0'),
 'age': 24000,
 'first_name': 'Gandalf',
 'last_name': 'the Grey'}
